In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import ToktokTokenizer
import re
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tapanpatro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from nltk.stem import PorterStemmer
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import SVC


In [3]:

train_df = pd.read_csv("train.csv")
train_df.head()


,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [4]:
# data_check = train_df

In [5]:
# data_check.head()

In [6]:
# df_check = data_check.drop(["ABSTRACT"], inplace=True)
# df_check = data_check.drop(['ABSTRACT'], axis=1)

In [7]:
# # This is i am writing after 3 time submission.. Just wanted to check is title have more good reason for labeling the data
# # 
# df_check.head(20)


In [8]:

test_df = pd.read_csv("test.csv")
test_df.head()


,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [9]:

sample_sub_df = pd.read_csv("sample_submission_UVKGLZE.csv")
sample_sub_df.head()


,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,0,0,0
1,20974,0,0,0,0,0,0
2,20975,0,0,0,0,0,0
3,20976,0,0,0,0,0,0
4,20977,0,0,0,0,0,0


In [10]:
sample_sub_df.shape

(8989, 7)

In [11]:


# train_df['content'] = train_df['TITLE']
# train_df['content'] = (2*train_df['ABSTRACT'])
# train_df['content'] = (5*train_df['TITLE'])+(2*train_df['ABSTRACT'])
train_df['content'] = train_df['TITLE'] + train_df['ABSTRACT']
# train_df['content'] = (5*train_df['TITLE']) + train_df['ABSTRACT']

In [12]:
train_df.drop(['ID', 'TITLE', 'ABSTRACT'], axis=1, inplace=True)
# train_df.drop(['ID', 'ABSTRACT'], axis=1, inplace=True)

In [13]:
train_df.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,content
0,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps P...
1,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation i...
2,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,0,0,1,0,0,0,A finite element approximation for the stochas...
4,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [14]:
print(train_df['content'][1])


Rotation Invariance Neural Network  Rotation invariance and translation invariance have great values in image
recognition tasks. In this paper, we bring a new architecture in convolutional
neural network (CNN) named cyclic convolutional layer to achieve rotation
invariance in 2-D symbol recognition. We can also get the position and
orientation of the 2-D symbol by the network to achieve detection purpose for
multiple non-overlap target. Last but not least, this architecture can achieve
one-shot learning in some cases using those invariance.



In [15]:

## Step - 1:: Clean the data
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"-", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = ''.join(c for c in text if not c.isnumeric())
    text = text.strip(' ')
    return text

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

In [16]:
token=ToktokTokenizer()

def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))



In [17]:

punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

train_df['content'] = train_df['content'].apply(lambda x: clean_text(x)) 
train_df['content'] = train_df['content'].apply(lambda x: clean_punct(x))




In [18]:
print(train_df['content'][1])


rotation invariance neural network rotation invariance and translation invariance have great values in image recognition tasks in this paper we bring a new architecture in convolutional neural network cnn named cyclic convolutional layer to achieve rotation invariance in d symbol recognition we can also get the position and orientation of the d symbol by the network to achieve detection purpose for multiple non overlap target last but not least this architecture can achieve one shot learning in some cases using those invariance


In [19]:

lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    
    stop_words = set(stopwords.words("english"))
    
    words=token.tokenize(text)
    
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))


ps = PorterStemmer() 

def stemWords(text):
    words=token.tokenize(text)
    listStem=[]
    for w in words:
        x=ps.stem(w)
        listStem.append(x)
    return ' '.join(map(str, listStem))


In [20]:

train_df['content'] = train_df['content'].apply(lambda x: lemitizeWords(x)) 
train_df['content'] = train_df['content'].apply(lambda x: stopWordsRemove(x))
# train_df['content'] = train_df['content'].apply(lambda x: stemWords(x))





In [21]:
print(train_df['content'][1])


rotation invariance neural network rotation invariance translation invariance great value image recognition task paper bring new architecture convolutional neural network cnn name cyclic convolutional layer achieve rotation invariance symbol recognition also get position orientation symbol network achieve detection purpose multiple non overlap target last least architecture achieve one shoot learn case use invariance


In [22]:
train_df.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,content
0,1,0,0,0,0,0,reconstruct subject specific effect map predic...
1,1,0,0,0,0,0,rotation invariance neural network rotation in...
2,0,0,1,0,0,0,spherical polyharmonics poisson kernels polyha...
3,0,0,1,0,0,0,finite element approximation stochastic maxwel...
4,1,0,0,1,0,0,comparative study discrete wavelet transform w...


In [23]:
train_df.columns

Index(['Computer Science', 'Physics', 'Mathematics', 'Statistics',
       'Quantitative Biology', 'Quantitative Finance', 'content'],
      dtype='object')

In [24]:
# train_df['Physics', "Mathematics"]
train_df.iloc[:, :6]

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0
...,...,...,...,...,...,...
20967,1,1,0,0,0,0
20968,0,1,0,0,0,0
20969,1,0,0,0,0,0
20970,0,0,1,1,0,0


In [25]:
y = train_df.iloc[:, :6] # train_df['Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']

In [26]:
total_size=train_df.shape[0]
train_size=int(0.80*total_size)

x_train=train_df.head(train_size)
x_test=train_df.tail(total_size - train_size)


In [27]:
print(total_size)
print(train_size)
print(total_size - train_size)

20972
16777
4195


In [28]:
# # y[0:16777,:]
# y[:16777].shape

In [29]:
y_train = y[0:train_size]
y_test = y[train_size:total_size]

In [30]:
print("Number of data points in train data :", y_train.shape)
print("Number of data points in test data :", y_test.shape)

Number of data points in train data : (16777, 6)
Number of data points in test data : (4195, 6)


# Part - 1

### TfidfVectorizer

In [31]:
# start = datetime.now()
# vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200000, smooth_idf=True, norm="l2", \
#                              tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
# x_train_multilabel = vectorizer.fit_transform(x_train['content'])
# x_test_multilabel = vectorizer.transform(x_test['content'])
# print("Time taken to run this cell :", datetime.now() - start)

In [32]:
# start = datetime.now()
# vectorizer = TfidfVectorizer(min_df=10, smooth_idf=True, norm="l2",
#                              tokenizer = lambda x: x.split(), sublinear_tf=True, ngram_range=(1,3))
# x_train_multilabel = vectorizer.fit_transform(x_train['content'])
# x_test_multilabel = vectorizer.transform(x_test['content'])
# print("Time taken to run this cell :", datetime.now() - start)

## 0.819

In [33]:
start = datetime.now()
vectorizer = TfidfVectorizer(min_df=1, smooth_idf=True, norm="l2", 
                             tokenizer = lambda x: x.split(), sublinear_tf=True, ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(x_train['content'])
x_test_multilabel = vectorizer.transform(x_test['content'])
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:22.159110


In [34]:
# start = datetime.now()
# vectorizer = TfidfVectorizer(min_df=5, smooth_idf=True, norm="l2", 
#                              tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
# x_train_multilabel = vectorizer.fit_transform(x_train['content'])
# x_test_multilabel = vectorizer.transform(x_test['content'])
# print("Time taken to run this cell :", datetime.now() - start)

# ## 82.7

In [35]:
# vectorizer.idf_

In [36]:
# # np.argsort(vectorizer.idf_[::-1])
# a = vectorizer.idf_.argsort()
# vectorizer.idf_[a]

In [37]:
# vectorizer.idf_.shape

In [38]:
# selected_idf_range = []
# # select_the_range = [1,5]

# heigher_range = 5
# lower_range = 1

# for idf_val in vectorizer.idf_:
#     if idf_val < heigher_range and idf_val > lower_range:
#         selected_idf_range.append(idf_val)

In [39]:
# np.array(selected_idf_range).shape

In [40]:

# indices = np.argsort(selected_idf_range)[::-1]
# features = vectorizer.get_feature_names()
# top_n = 100
# # top_features = ["{}::{}".format(features[i],selected_idf_range[np.array(selected_idf_range).argsort()][i]) for i in indices[:top_n]]
# top_features = ["{}::{}".format(features[i],selected_idf_range[i]) for i in indices[:top_n]]

# print (top_features)

In [41]:
# tfidf = x_train_multilabel.todense()

# means = np.nanmean(tfidf, axis=0)
# # convert it into a dictionary for later lookup
# means = dict(zip(tfidf_vectorizer.get_feature_names(), means.tolist()[0]))

# tfidf = tfidf_vectorizer_vectors.todense()
# # Argsort the full TFIDF dense vector
# ordered = np.argsort(tfidf*-1)
# words = tfidf_vectorizer.get_feature_names()

In [42]:
# top_k = 5
# for i, doc in enumerate(docs):
#     result = { }
#     # Pick top_k from each argsorted matrix for each doc
#     for t in range(top_k):
#         # Pick the top k word, find its average tfidf from the
#         # precomputed dictionary using nanmean and save it to later use
#         result[words[ordered[i,t]]] = means[words[ordered[i,t]]]
#     print (result )


In [43]:
# def maketfidfw2vList(feature, xEassyArray):
    
#     # S = ["abc def pqr", "def def def abc", "pqr pqr def"]
#     tfidf_model = TfidfVectorizer()
#     tfidf_model.fit(X_train['{}'.format(feature)])
#     # we are converting a dictionary with word as a key, and the idf as a value
#     dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
#     tfidf_words = set(tfidf_model.get_feature_names())
    
    
    
#     tfidf_w2v_vectors_project_text = []; # the avg-w2v for each sentence/review is stored in this list
#     for sentence in tqdm(xEassyArray): # for each review/sentence
#         vector = np.zeros(300) # as word vectors are of zero length
#         tf_idf_weight =0; # num of words with a valid vector in the sentence/review
#         for word in sentence.split(): # for each word in a review/sentence
#             if (word in glove_words) and (word in tfidf_words):
#                 vec = model[word] # getting the vector for each word
#                 # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
#                 tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
#                 vector += (vec * tf_idf) # calculating tfidf weighted w2v
#                 tf_idf_weight += tf_idf
#         if tf_idf_weight != 0:
#             vector /= tf_idf_weight
#         tfidf_w2v_vectors_project_text.append(vector)
#     return tfidf_w2v_vectors_project_text

#### Count

In [44]:
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(min_df=10,ngram_range=(1,10)) # , max_features=5000
# vectorizer.fit(x_train['content'])

# x_train_multilabel = vectorizer.transform(x_train['content'])
# x_test_multilabel = vectorizer.transform(x_test['content'])

In [45]:
# x_train_multilabel.todense()

In [46]:
print("Dimensions of train data X:",x_train_multilabel.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",x_test_multilabel.shape,"Y:",y_test.shape)

Dimensions of train data X: (16777, 2500833) Y : (16777, 6)
Dimensions of test data X: (4195, 2500833) Y: (4195, 6)


### Model - OneVsRestClassifier

In [47]:
# this will be taking so much time try not to run it, download the lr_with_equal_weight.pkl file and use to predict
# This takes about 6-7 hours to run.
# classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1) # 1st submission
# classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.0001, penalty='l2'), n_jobs=-1) # 2nd Submission
classifier = OneVsRestClassifier(LinearSVC(penalty='l2', loss='hinge'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)
predictions = classifier.predict(x_test_multilabel)

print("accuracy :",metrics.accuracy_score(y_test,predictions))
print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
print("micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(y_test,predictions))
print("Precision recall report :\n",metrics.classification_report(y_test, predictions))


accuracy : 0.6693682955899881
macro f1 score : 0.6874392706139854
micro f1 scoore : 0.8320089744788258
hamming loss : 0.07139451728247914
Precision recall report :
               precision    recall  f1-score   support

           0       0.80      0.93      0.86      1733
           1       0.93      0.85      0.89      1200
           2       0.87      0.77      0.82      1127
           3       0.72      0.88      0.79      1054
           4       0.77      0.10      0.18        99
           5       0.83      0.46      0.60        54

   micro avg       0.82      0.84      0.83      5267
   macro avg       0.82      0.66      0.69      5267
weighted avg       0.83      0.84      0.83      5267
 samples avg       0.84      0.87      0.84      5267



/Users/tapanpatro/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
## Basic title + abstact +steming -> 0.81432 --> 0.828284
## Only Title have 0.74 micro f1 score <==> Steming -> 0.75
## So lets try with weighted title 2x title + abstaract -> 0.81 ==> 0.81 -> 0.8265
## Now 5 x Title + anstaract + steming-> 0.80 -> 0.85
## Only abstarct -> 0.823 -> 0.823
## 2 x abstarct -> 0.823
## title+ 2 abst -> 0.8263

## Witout steming
## --> 0.825
## 5xtitle+abs --> 0.81 --> uploaded 0.826
## 10title_abs --> 0.80 --> 0.8222
## 4titl+abs --> 0.8215 --> 0.824



## Count Vectorizer
## title_abs --> 0.76

### New Tfvect
## 5t_a + new tfidf -> 0.820
## ta + tfidf -> 0.81

## New tfidf, mindif=1, l2, (1,2 gram), 82.7 --> 82.5
## 10, l2, 1,3 -> 81.
## 1, l2, 1,3 -> 83.1 --> 82.7 ********************************************************************************


## New 1,l2, 1-3 Linear SVC - multi_class : 83.1 --> 83.0
## 83.2 -> 78


#### Cleaning and processing Test Data

In [48]:
test_df.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [49]:
test_df['content'] = test_df['TITLE'] + test_df['ABSTRACT']

In [50]:
print(test_df['content'][1])

Laboratory mid-IR spectra of equilibrated and igneous meteorites. Searching for observables of planetesimal debris  Meteorites contain minerals from Solar System asteroids with different
properties (like size, presence of water, core formation). We provide new
mid-IR transmission spectra of powdered meteorites to obtain templates of how
mid-IR spectra of asteroidal debris would look like. This is essential for
interpreting mid-IR spectra of past and future space observatories, like the
James Webb Space Telescope. We show that the transmission spectra of wet and
dry chondrites, carbonaceous and ordinary chondrites and achondrite and
chondrite meteorites are distinctly different in a way one can distinguish in
astronomical mid-IR spectra. The two observables that spectroscopically
separate the different meteorites groups (and thus the different types of
parent bodies) are the pyroxene-olivine feature strength ratio and the peak
shift of the olivine spectral features due to an increase in

In [51]:
test_df['content'] = test_df['content'].apply(lambda x: clean_text(x)) 
test_df['content'] = test_df['content'].apply(lambda x: clean_punct(x))

In [52]:
test_df['content'] = test_df['content'].apply(lambda x: lemitizeWords(x)) 
test_df['content'] = test_df['content'].apply(lambda x: stopWordsRemove(x))

In [53]:
print(test_df['content'][1])

laboratory mid ir spectra equilibrate igneous meteorites search observables planetesimal debris meteorites contain minerals solar system asteroids different properties like size presence water core formation provide new mid ir transmission spectra powder meteorites obtain templates mid ir spectra asteroidal debris would look like essential interpret mid ir spectra past future space observatories like jam webb space telescope show transmission spectra wet dry chondrites carbonaceous ordinary chondrites achondrite chondrite meteorites distinctly different way one distinguish astronomical mid ir spectra two observables spectroscopically separate different meteorites group thus different type parent body pyroxene olivine feature strength ratio peak shift olivine spectral feature due increase iron concentration olivine


In [54]:
test_df_multilabel = vectorizer.transform(test_df['content'])

In [55]:
predictions_test = classifier.predict(test_df_multilabel)

In [56]:
predictions_test

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [57]:
predictions_test_frame = pd.DataFrame(predictions_test, columns=['Computer Science', 'Physics', 'Mathematics', 'Statistics','Quantitative Biology', 'Quantitative Finance'])

In [58]:
predictions_test_frame.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,0,0
4,1,0,0,0,0,0


In [59]:
predictions_test_frame.shape

(8989, 6)

In [60]:
submission = pd.concat([test_df, predictions_test_frame], axis=1)

In [61]:
submission.head()

,ID,TITLE,ABSTRACT,content,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...,close form marginal likelihood gamma poisson m...,0,0,0,1,0,0
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...,laboratory mid ir spectra equilibrate igneous ...,0,1,0,0,0,0
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...,case static amsdu aggregation wlans frame aggr...,1,0,0,0,0,0
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...,gaia eso survey inner disk intermediate age op...,0,1,0,0,0,0
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...,witness function versus interpretation functio...,1,0,0,0,0,0


## As requirement removing unwanted columns

In [62]:
submission.drop(["TITLE", "ABSTRACT", "content"],axis=1, inplace=True)

In [63]:
submission.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [64]:
submission.shape

(8989, 7)

In [65]:
submission.to_csv("submission_83try.csv", index=False)

# Part - 2 

In [ ]:
# this will be taking so much time try not to run it, download the lr_with_equal_weight.pkl file and use to predict
# This takes about 6-7 hours to run.
classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.0001, penalty='l2'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)
predictions = classifier.predict(x_test_multilabel)

print("accuracy :",metrics.accuracy_score(y_test,predictions))
print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
print("micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'))
print("hamming loss :",metrics.hamming_loss(y_test,predictions))
print("Precision recall report :\n",metrics.classification_report(y_test, predictions))


In [ ]:
def print_score(y_pred, clf):
#     print("Clf: ", clf.__class__.__name__)
#     print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
#     print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
     
    print("accuracy :",metrics.accuracy_score(y_test,predictions))
    print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
    print("micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'))
    print("hamming loss :",metrics.hamming_loss(y_test,predictions))
#     print("Precision recall report :\n",metrics.classification_report(y_test, predictions))
    print("---")


In [ ]:
# dummy = DummyClassifier()
# sgd = SGDClassifier()
# lr = LogisticRegression()
# mn = MultinomialNB()
# svc = LinearSVC() # 82.25
# perceptron = Perceptron()
# pac = PassiveAggressiveClassifier()
mlpc = MLPClassifier()
# random = RandomForestClassifier()

# for eachclf in [dummy, sgd, lr, mn, svc, perceptron, pac, mlpc, random]:
#     clf = OneVsRestClassifier(eachclf)
#     clf.fit(x_train_multilabel, y_train)
#     y_pred = clf.predict(x_test_multilabel)
#     print("--------------")
#     print(clf)
#     print_score(y_pred, eachclf)


# classifier = OneVsRestClassifier(LinearSVC(penalty='l2', loss='hinge'), n_jobs=-1)
# 82.5845434168




# this will be taking so much time try not to run it, download the lr_with_equal_weight.pkl file and use to predict
# This takes about 6-7 hours to run.
classifier = OneVsRestClassifier(SVC(kernel="linear", gamma='auto'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)
predictions = classifier.predict(x_test_multilabel)



# model_to_set = OneVsRestClassifier(SVC(kernel="poly"))




print("accuracy :",metrics.accuracy_score(y_test,predictions))
print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
print(" ")
print("\033[1m micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'),"\033[0m")
print(" ")
print("hamming loss :",metrics.hamming_loss(y_test,predictions))
print("Precision recall report :\n",metrics.classification_report(y_test, predictions))

